In [1]:
from astropy import units as u
from astropy.time import Time, TimeDelta

from poliastro.util import norm, time_range
from poliastro.frames import Planes
from poliastro.ephem import Ephem
from poliastro.bodies import Sun, Earth

from poliastro.twobody.orbit import Orbit
from poliastro.twobody.sampling import EpochsArray

from astropy.coordinates import solar_system_ephemeris

import numpy as np

from stations import dss25, dss34
from tracking import Tracking
from util import describe_orbit, describe_state, describe_trajectory

solar_system_ephemeris.set("de440")
start_epoch = Tracking.NEAR_GOLDSTONE_END.value
end_epoch = Tracking.NEAR_CANBERRA_START.value

In [2]:
near_start_ephem = Ephem.from_horizons("NEAR", start_epoch, attractor=Earth, plane=Planes.EARTH_EQUATOR)
near_start_rv = near_start_ephem.rv(start_epoch)
describe_state(near_start_rv, dss25, start_epoch)

near_orbit = Orbit.from_vectors(Earth, near_start_rv[0], near_start_rv[1], start_epoch)
describe_orbit(near_orbit)


:AT: 1998-01-23 06:14:55.600
Geocentric distance: 37471.700215992445 km speed: 8.258344936971223 km / s
From Goldstone-25 : range  31576.174442159892 km , range rate  -7.684513660227914 km / s

::ORBIT::
Plane: Planes.EARTH_EQUATOR
Inclination: 107.97266698121356 deg
Eccentricity: 1.813357091534396
Semilatus rectum: 19437.250829758486 km
Semimajor axix: -8494.322039490264 km
Periapse radius: 6908.917068596319 km , altitude: 530.7804685963192 km


In [23]:
sampling_interval = 180
flyby_epochs = start_epoch + (np.arange(0, (end_epoch-start_epoch)/(1*u.s), sampling_interval) << u.s)
print("Start and end epochs:", start_epoch, flyby_epochs[-1], "[", flyby_epochs.size, "]")

Start and end epochs: 1998-01-23 06:14:55.600 1998-01-23 09:53:55.600 [ 74 ]


In [24]:
near_ephem = near_orbit.to_ephem(EpochsArray(flyby_epochs))
describe_trajectory(near_ephem, dss34)


::TRAJECTORY::
Start and end ranges: 37309.07130107391 km 73145.82143191747 km
Start and end radial speeds: -7.97119272554438 km / s 7.3216176182785535 km / s
Closest: 11326.436162795784 km speed 12.740147538216107 km / s  at  1998-01-23 07:23:55.600
Deflection: 64.92614895100142 deg


In [25]:
predict_end_rv = near_ephem.rv(end_epoch)
describe_state(predict_end_rv, dss34, end_epoch)


:AT: 1998-01-23 09:53:55.600
Geocentric distance: 74715.16140743632 km speed: 7.5891622264313146 km / s
From Canberra-34 : range  73145.82143191747 km , range rate  7.3216176182785535 km / s


In [26]:
near_end_ephem = Ephem.from_horizons("NEAR", end_epoch, attractor=Earth, plane=Planes.EARTH_EQUATOR)
near_end_rv = near_end_ephem.rv(end_epoch)
describe_state(near_end_rv, dss34, end_epoch)


:AT: 1998-01-23 09:53:55.600
Geocentric distance: 74714.76607137504 km speed: 7.589138197919106 km / s
From Canberra-34 : range  73147.47485388452 km , range rate  7.32170013250344 km / s


In [27]:
print(norm(predict_end_rv[0]-near_end_rv[0]) << u.km, norm(predict_end_rv[1]-near_end_rv[1]) << (u.m/u.s))

35.71831820614084 km 3.4529739544187095 m / s
